<a href="https://colab.research.google.com/github/geekevgin/-Python/blob/main/HW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Исследования проводились на бактретии Pseudomonas aeruginosa. Усиливает ли медь антибиотикорезистентность у резистентных видов и/или вызывает устойчивость
# у чувствительных мо.


In [1]:
!apt-get -y install fastqc bwa rna-star

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  default-jre default-jre-headless fonts-dejavu-core fonts-dejavu-extra
  libapache-pom-java libargs4j-java libatk-wrapper-java
  libatk-wrapper-java-jni libcommons-compress-java libcommons-io-java
  libcommons-jexl2-java libcommons-lang3-java libcommons-logging-java
  libcommons-math3-java libcommons-parent-java libfindbin-libs-perl
  libfontenc1 libhts3 libhtscodecs2 libhtsjdk-java libjbzip2-java
  libjson-simple-java libngs-java libngs-sdk-dev libngs-sdk2 libsis-base-java
  libsis-base-jni libsis-jhdf5-java libsis-jhdf5-jni libsnappy-java
  libsnappy-jni libxkbfile1 libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
Suggested packages:
  samtools libxz-java libcommons-io-java-doc libcommons-lang3-java-doc
  libavalon-framework-java libcommons-logging-java-doc
  libexcalibur-logkit-java liblog4j1.2-java picard-tools libjson-simple-doc
  mes

In [2]:
google_drive_url = "https://drive.google.com/uc?export=download&id="

In [3]:
reads_file_id = "1N4uHhyaTuKFd-vyvNNM_e9WisI9w3INO"
reads_filename = 'SRR12905311.fastq.gz'



In [4]:
# Архив с ридами скачать сложнее, так как надо кликнуть согласие на пропуск антивирусной проверки
!wget --save-cookies cookies.txt "$google_drive_url$reads_file_id" -O- | sed -rn "s/.*confirm=([0-9A-Za-z_]+).*/\1/p" > confirm.txt
!wget --load-cookies cookies.txt -O $reads_filename "$google_drive_url$reads_file_id&confirm=$(<confirm.txt)"

--2023-08-05 18:10:53--  https://drive.google.com/uc?export=download&id=1N4uHhyaTuKFd-vyvNNM_e9WisI9w3INO
Resolving drive.google.com (drive.google.com)... 142.250.107.139, 142.250.107.113, 142.250.107.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.107.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&continue=https://drive.google.com/uc?export%3Ddownload%26id%3D1N4uHhyaTuKFd-vyvNNM_e9WisI9w3INO&followup=https://drive.google.com/uc?export%3Ddownload%26id%3D1N4uHhyaTuKFd-vyvNNM_e9WisI9w3INO [following]
--2023-08-05 18:10:53--  https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&continue=https://drive.google.com/uc?export%3Ddownload%26id%3D1N4uHhyaTuKFd-vyvNNM_e9WisI9w3INO&followup=https://drive.google.com/uc?export%3Ddownload%26id%3D1N4uHhyaTuKFd-vyvNNM_e9WisI9w3INO
Resolving accounts.google.com (accounts.google.com)... 142.250.107.84, 2607:f8b0:400

In [5]:
annotation_file_id = "1cip1xehhZZunD6KiJX3PMan1qman2ZX2"
annotation_filename = "genomic.gtf"

In [6]:
!wget -O $annotation_filename "$google_drive_url$annotation_file_id"

--2023-08-05 18:10:59--  https://drive.google.com/uc?export=download&id=1cip1xehhZZunD6KiJX3PMan1qman2ZX2
Resolving drive.google.com (drive.google.com)... 142.250.107.139, 142.250.107.113, 142.250.107.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.107.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&continue=https://drive.google.com/uc?export%3Ddownload%26id%3D1cip1xehhZZunD6KiJX3PMan1qman2ZX2&followup=https://drive.google.com/uc?export%3Ddownload%26id%3D1cip1xehhZZunD6KiJX3PMan1qman2ZX2 [following]
--2023-08-05 18:10:59--  https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&continue=https://drive.google.com/uc?export%3Ddownload%26id%3D1cip1xehhZZunD6KiJX3PMan1qman2ZX2&followup=https://drive.google.com/uc?export%3Ddownload%26id%3D1cip1xehhZZunD6KiJX3PMan1qman2ZX2
Resolving accounts.google.com (accounts.google.com)... 142.250.107.84, 2607:f8b0:400

In [7]:
reference_file_id = "1ZEyGjhEPwY6JlKYQPvB-XtkpRYU_olEq"
reference_filename = "GCF_000006765.1_ASM676v1_genomic.fna"

In [8]:
!wget -O $reference_filename "$google_drive_url$reference_file_id"

--2023-08-05 18:11:05--  https://drive.google.com/uc?export=download&id=1ZEyGjhEPwY6JlKYQPvB-XtkpRYU_olEq
Resolving drive.google.com (drive.google.com)... 142.250.107.139, 142.250.107.113, 142.250.107.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.107.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&continue=https://drive.google.com/uc?export%3Ddownload%26id%3D1ZEyGjhEPwY6JlKYQPvB-XtkpRYU_olEq&followup=https://drive.google.com/uc?export%3Ddownload%26id%3D1ZEyGjhEPwY6JlKYQPvB-XtkpRYU_olEq [following]
--2023-08-05 18:11:05--  https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&continue=https://drive.google.com/uc?export%3Ddownload%26id%3D1ZEyGjhEPwY6JlKYQPvB-XtkpRYU_olEq&followup=https://drive.google.com/uc?export%3Ddownload%26id%3D1ZEyGjhEPwY6JlKYQPvB-XtkpRYU_olEq
Resolving accounts.google.com (accounts.google.com)... 142.250.107.84, 2607:f8b0:400

In [9]:
!gunzip $reads_filename
!ls -al


gzip: SRR12905311.fastq.gz: not in gzip format
total 1728
drwxr-xr-x 1 root root   4096 Aug  5 18:11 .
drwxr-xr-x 1 root root   4096 Aug  5 18:07 ..
drwxr-xr-x 4 root root   4096 Aug  3 13:44 .config
-rw-r--r-- 1 root root      0 Aug  5 18:10 confirm.txt
-rw-r--r-- 1 root root    195 Aug  5 18:10 cookies.txt
drwx------ 5 root root   4096 Aug  5 18:10 drive
-rw-r--r-- 1 root root 580507 Aug  5 18:11 GCF_000006765.1_ASM676v1_genomic.fna
-rw-r--r-- 1 root root 580556 Aug  5 18:10 genomic.gtf
drwxr-xr-x 1 root root   4096 Aug  3 13:45 sample_data
-rw-r--r-- 1 root root 580523 Aug  5 18:10 SRR12905311.fastq.gz


In [ ]:
reads_filename = reads_filename.removesuffix(".gz")
reads_filename

In [ ]:
!fastqc $reads_filename

In [10]:
!bwa index $reference_filename

[bwa_index] Pack FASTA... 0.11 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.12 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.10 sec
[bwa_index] Construct SA from BWT and Occ... 0.06 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index GCF_000006765.1_ASM676v1_genomic.fna
[main] Real time: 0.477 sec; CPU: 0.399 sec


In [12]:
alignment_filename = reads_filename.removesuffix(".fastq") + "_alignment.sam"


In [13]:
!bwa mem $reference_filename $reads_filename > $alignment_filename

[bns_restore_core] Parse error reading GCF_000006765.1_ASM676v1_genomic.fna.amb



In [ ]:
!samtools stats $alignment_filename | head -n 30

In [ ]:
!mkdir -p star_genome

In [ ]:
!STAR --runMode genomeGenerate --genomeDir star_genome --genomeFastaFiles $reference_filename

In [ ]:
!STAR --runMode alignReads --genomeDir star_genome --quantMode GeneCounts \
	  --outFileNamePrefix map_prefix --sjdbGTFfile $annotation_filename \
	  --readFilesIn $reads_filename